In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
df = pd.read_excel('Training-testing-TUI.xlsx',sheet_name = 'Sheet1')
df.rename(columns = {'Date - Month':'Date'},inplace = True)
df.rename(columns = {'Current Status':'Current_Status'},inplace = True)
df = df.query("Current_Status not in ('Never Started','Leave of Absence')")
df.rename(columns = {'Current_Status':'Current Status'},inplace = True)
df.shape

(599, 82)

In [410]:
df['Department'].value_counts()

Representative    2999
Name: Department, dtype: int64

In [370]:
values = df.query("Date in ('January' ,'February' , 'December')")
values['Position'].value_counts()

Representative    2209
Name: Position, dtype: int64

In [3]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [4]:
def preprocess_input(df):
    df = df.copy()
    
    # Filtring out null working hours
    df = df.query("Date in ('January' ,'February' , 'December')")
    df = df.fillna(0)
    # droping one value columns or null
    #df = df.drop(['Manager','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client','Market','Language','Location','Contract Type','Department','Termination Type','Termination Reason'], axis=1)
    df = df.drop(['University','Nationality','Address 1','Major'
                  ,'Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Termination Type','Termination Reason',
                  'Sum of Internal Moves','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %',
                  'Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation','Department'], axis=1)
   
    # Binary-encode binary columns
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0,'Never Started':0,'Leave of Absence':0})
    # One-hot encoding
    for column in ['Program','Current Civil Status','District','Manager']:
        df = onehot_encode(df, column=column)
    
    
    
    # Split df into X and y
    y = df['Current Status']
    df = df.reindex(labels=Predict.columns,axis=1)
    df = df.fillna(0)
    X = df.drop('Current Status', axis=1)
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    #scaler = StandardScaler()
    #scaler.fit(X_train)
    #X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    #X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [5]:
def preprocess_input2(df):
    df = df.copy()
    # Filtring out null working hours
    df = df.query("Date == 'March'")
    df = df.fillna(0)
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    #df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    #df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0})
    df = df.drop(['University','Nationality','Major','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Termination Type','Termination Reason',
                  'Sum of Internal Moves','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %','Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation','Department'], axis=1)
 
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    # One-hot encoding
    for column in ['Program','Current Civil Status','District','Manager']:
        df = onehot_encode(df, column=column)
    return df

In [6]:
Predict = preprocess_input2(df)
X_train, X_test, y_train, y_test = preprocess_input(df)
X_train

,Date - Year,Date,Sum of Salary,Gender.,Age,Graduate / Undergrad,AHT,Sum of Handled Transactions,Call Log %,Sum of Total Surveys,Agent CSAT,Agent CSAT %,Service CSAT,Service CSAT %,CES %,FCR %,QA Monitored,"QA Critical Pass % (EUC, BC, CC, ZTP)",QA EUC Pass %,QA BC Pass %,QA CC Pass %,QA ZTP Pass %,Other Monitored Transactions,Positive Coaching,Sum of Working Hours On Premises,Sum of Working Hours WAHA,Sum of Over-Time Days,Sum of Working During Holdidays,Occupancy %,ABS %,Sum of Paid Leaves,Sum of Unpaid Leaves,Sum of Sick-Leaves,Sum of No Shows,Sum of Attrition,Approved Actions,Approved Red,Canceled Red,Approved Black,Canceled Black,Program_TUI Travel PLAZA New Hire,Program_TUI Travel PLAZA Voice Agent,Current Civil Status_Single,Current Civil Status_Unknown,District_-,District_6th of October,District_Abdeen,District_Abu Kibir,District_Ain Shams,District_Al Agouzah,District_Al Kasr Al Aini,District_Al Manial,District_Al Omrania,District_Al Rehab,District_Al-Haram,District_Ard el lewa,District_Berkat El Sabaa,District_Boulaq El Dakrour,District_Daher,District_El Basatin,District_El Marg,District_El Nozha,District_El Sahel,District_El Shorouk,District_El Ubour,District_El-Mahalla El-Kubra,District_El-Mansheya,District_El-Nozha,District_El-Salam,District_El-Zaytoun,District_Embaba,District_Fifth Settlement,District_First Settlement,District_Fisal,District_Glim,District_Hadaeq Al Qubbah,District_Hadayek al-Qubba,District_Heliopolis,District_Helwan,District_Ibn Sendar,District_Katameya,District_Maadi,District_Madinaty,District_Mansoura,District_Masr Al Jadida,District_Mohandessin,District_Mokattam,District_Nasr City,District_Nozha,District_Rod El Farag,District_Saraya Al Koba,District_Sidi Jaber,District_Tanta,District_Zagazig,"Manager_Abaza, Aya A.","Manager_Abdelhady, Mariam W.","Manager_Diaaeldin, Youssef O.","Manager_Fathy, Omar A.","Manager_Hikel, Rasha H.","Manager_Ibrahim, Hidi H.","Manager_Kamel, Omar M.","Manager_Khalil, Fatma H.","Manager_Metwaly, Ahmed R.","Manager_Omer, Sham A.","Manager_Othman, Hassan N.","Manager_Salah, Khaled S.","Manager_Youssef, Moamen M."
412,2022,2,6000,1,37,2,728.299728,734,0.976839,17,6.764706,0.529412,5.705882,0.470588,0.500000,0.437500,8.0,0.75,1.0,1.0,0.75,0.0,32.0,0.0,364.361944,0,0.0,0.0,0.969236,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0
28,2021,0,6000,0,18,0,752.626214,412,0.975728,13,6.461538,0.461538,5.692308,0.461538,0.461538,0.461538,7.0,1.00,1.0,1.0,1.00,0.0,16.0,0.0,336.577222,0,0.0,0.0,0.817040,0.210526,0.0,0.0,10.0,10.0,0.0,1.0,1.0,1.0,0.0,0.0,0,1,1,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0.0,1,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0
16,2021,0,6000,1,31,2,905.000000,20,1.000000,2,7.500000,0.500000,5.000000,0.500000,0.500000,0.500000,1.0,1.00,1.0,1.0,1.00,0.0,7.0,0.0,230.499722,0,0.0,0.0,0.733596,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0
520,2022,2,6000,1,25,2,680.111111,9,0.777778,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,6.0,0.0,101.088333,0,0.0,0.0,0.999831,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,1,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,1,0.0,0.0,0,0,0,0.0,0,0,0,0.0,0
405,2022,2,6000,1,26,2,833.474359,312,0.967949,13,9.153846,0.923077,8.846154,0.846154,0.750000,0.923077,7.0,1.00,1.0,1.0,1.00,0.0,37.0,0.0,239.629167,0,0.0,0.0,0.968046,0.316552,0.0,0.0,0.0,20.0,0.0,3.0,3.0,0.0,0.0,0.0,1,0,1,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,1,0.0,0,0,0,0.0,0

In [7]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")
    
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))
    
Predict = Predict.drop(columns=['Current Status'], axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                   Logistic Regression: 73.17%
                   K-Nearest Neighbors: 65.85%
                         Decision Tree: 59.76%
Support Vector Machine (Linear Kernel): 68.29%
   Support Vector Machine (RBF Kernel): 68.29%
                        Neural Network: 68.29%
                         Random Forest: 75.61%
                     Gradient Boosting: 75.61%


In [338]:
Predict = Predict.drop(columns=['Test'], axis = 1)

In [8]:
log_mod = LogisticRegression()
log_mod.fit(X_train, y_train)
Predict['Test'] = log_mod.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.7591240875912408,
 1    104
 0     33
 Name: Test, dtype: int64)

In [363]:
Predict = Predict.drop(columns=['Test'], axis = 1)
log_mod = KNeighborsClassifier()
log_mod.fit(X_train, y_train)
Predict['Test'] = log_mod.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9455696202531646,
 1    747
 0     43
 Name: Test, dtype: int64)

In [350]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = DecisionTreeClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.8645569620253165,
 1    683
 0    107
 Name: Test, dtype: int64)

In [351]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = SVC()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(1.0,
 1    790
 Name: Test, dtype: int64)

In [352]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = MLPClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9848101265822785,
 1    778
 0     12
 Name: Test, dtype: int64)

In [353]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = RandomForestClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9620253164556962,
 1    760
 0     30
 Name: Test, dtype: int64)

In [354]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = GradientBoostingClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9582278481012658,
 1    757
 0     33
 Name: Test, dtype: int64)

In [ ]:
df['']

In [9]:
df.to_excel('TUI Risk List.xlsx',index = False)